In [1]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from ces_fun_util import *
#from fun_utils_eq import *

In [2]:
## parameter values
alpha = 0.85           # labor share parameter in manufacturing
#alpha = 0.2
theta = 4             # scopevec for comparative advantage
sigma = 1.2      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1.2  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar  #Foreign's elasticity of demand for embodied energy
#beta = 2.274853
#gamma= 0.784877595
beta=1.892412
gamma=0.807998928
epsilonS = 0.5  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [3]:
## scenario switch
case=3 # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0 # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [4]:
ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)

In [5]:
class taxrow_eq:
    def __init__(self, pe, te, tb_mat, df, tax_scenario, varphi, ParaList):
        self.df = df
        self.tax_scenario = tax_scenario
        self.varphi = varphi
        self.ParaList = ParaList
        self.prop = 1
        self.te = te
        self.pe = pe
        self.tb_mat = tb_mat

    ## computes equilibrium price and taxes. Also computes other equilibrium values (ie consumption, production, value of exports/imports)
    ## and stores them in self. 
    def opt_tax(self):
        tax_scenario = self.tax_scenario
        varphi = self.varphi
        df = self.df
        te = varphi
        self.region = df['region_scenario']
        pes = np.append([self.pe], np.arange(0.1, 2.5, 0.2))
        tbs = np.append([self.tb_mat[0]], np.arange(0,1.5,0.2))
        props = np.append([self.tb_mat[1]], np.arange(0,1.1,0.5))
        self.conv = 1
        if (tax_scenario['tax_sce'] == 'global'):
            tbs = [0]
        
        if tax_scenario['tax_sce'] != 'purete' and tax_scenario['tax_sce'] != 'EP_hybrid':
            for prop in props:
                for tb in tbs:
                    for pe in pes:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, 1)],
                               method='nelder-mead', args=(te), tol=0.000001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            #print(res.fun, df['region_scenario'], tax_scenario['tax_sce'], varphi)
            
        pes = np.append([self.pe], np.arange(0.1, 2, 0.5))
        props = np.append([self.tb_mat[1]], np.arange(0,2,0.5))
        if tax_scenario['tax_sce'] == 'EP_hybrid':
            for pe in pes:
                for tb in tbs:
                    for prop in props:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, np.inf)],
                               method='nelder-mead', args=(te), tol=0.000001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            
            
        ## not implemented yet
        if tax_scenario['tax_sce'] == 'Baseline':
            self.tb = 0
            self.prop = 1
            self.te = 0
        
        elif tax_scenario['tax_sce'] == 'global':
            self.tb = 0
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'PC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = self.tb

        elif tax_scenario['tax_sce'] == 'EPC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'EP_hybrid':
            self.tb = opt_val[1]
            self.te = opt_val[2]
            self.prop = self.te #- self.tb

        elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'puretc' or tax_scenario[
            'tax_sce'] == 'puretp' or tax_scenario['tax_sce'] == 'EC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]

            if tax_scenario['tax_sce'] == 'puretc' or tax_scenario['tax_sce'] == 'puretp':
                self.te = self.tb

            elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'EC_hybrid':
                self.te = self.varphi

        elif tax_scenario['tax_sce'] == 'purete':
            res = minimize(self.minuswelfare, [self.pe, self.te], bounds=[(0.001, np.inf), (0, np.inf)], method='nelder-mead', args=(0),
                           tol=0.000001, options={'maxfev': 100000})
            
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            #print(res.fun)
            self.te = res.x[1]
            tb_mat = [0, 1]
            self.tb = tb_mat[0]
            self.prop = tb_mat[1]
        #print(tax_scenario['tax_sce'], res)

    def minuswelfare(self, p, te):
        pe = p[0]
        tb_mat = p[1:]
        varphi = self.varphi
        tax_scenario = self.tax_scenario
        ParaList = self.ParaList
        df = self.df

        if tax_scenario['tax_sce'] == 'purete':
            te = p[1]
            tb_mat = [0, 1]

        obj = self.comp_obj(pe, te, tb_mat, varphi, tax_scenario, ParaList, df)

        return obj

    ## compute the objective value, currently the objective is to minimize negative welfare
    ## also saves optimal results in self.
    def comp_obj(self, pe, te, tb_mat, varphi, tax_scenario, Paralist, df):

        alpha, theta, sigma, sigmastar, epsilonD, epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
        ## compute extraction tax, and jbar's
        te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)

        jxbar_prime = jxbar_hat * df['jxbar']
        jmbar_prime = jmbar_hat * df['jmbar']
        j0_prime = j0_hat * df['jxbar']

        jvals = (j0_hat, j0_prime, jxbar_hat, jxbar_prime, jmbar_hat, jmbar_prime)

        # if te is too large, Home stop producing
        petbte = pe + tb_mat[0] - te
        z = pe + tb_mat[0] >= te
        petbte = petbte * z

        Qe_prime, Qestar_prime = compute_qe(tax_scenario, petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

        Cey_prime, Cex1_prime, Cex2_prime, Cex_prime, Cem_prime, CeFF_prime, Cex_hat, Cex1_hat, Cem_hat = comp_ce(
            pe, tb_mat, jvals, ParaList, df, tax_scenario)

        Ge_prime = Cey_prime + Cex_prime
        Gestar_prime = CeFF_prime + Cem_prime
        Ce_prime = Cey_prime + Cem_prime
        Cestar_prime = CeFF_prime + Cex_prime
        Qeworld_prime = Qe_prime + Qestar_prime

        consvals = (Cey_prime, Cex1_prime, Cex2_prime, Cex_prime, 
                    Cem_prime, CeFF_prime, Cex_hat, Cex1_hat, Cem_hat,
                    Ce_prime, Cestar_prime)

        Vgy, Vgystar, Vgx1_prime, Vgx2_prime, Vgx_prime, Vgm_prime, Vgx, Vgm = comp_vg(pe, tb_mat, jvals, consvals,
                                                                                           df, tax_scenario, ParaList)

        pai_g = Vgx - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha)
        subsidy_ratio = 1- ((1-jxbar_prime) * j0_prime / ((1-j0_prime) * jxbar_prime))**(1/theta)

        ## compute Ve values
        Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, consvals, tax_scenario)

        ## compute more Vg values
        vgvals = Vgx_prime, Vgm_prime
        Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, consvals, vgvals, jvals, ParaList, df, tax_scenario)

        Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, Ge_prime, Gestar_prime, consvals, ParaList, df, tax_scenario)

        leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)

        lgvals = Lg, Lgstar, Lg_prime, Lgstar_prime
        vgvals = Vg, Vgstar, Vg_prime, Vgstar_prime
        delta_Le, delta_Lestar, delta_U = comp_delta(lgvals, vgvals, Qeworld_prime, df, jvals, pe, petbte, tb_mat, tax_scenario, varphi, ParaList)

        chg_extraction, chg_production, chg_consumption, chg_Qeworld = comp_chg(df, Qestar_prime, Gestar_prime,
                                                                                Cestar_prime, Qeworld_prime)

        welfare = delta_U / Vg * 100
        welfare_noexternality = (delta_U + varphi * (Qeworld_prime - df['Qeworld'])) / Vg * 100
        
        self.results = assign_val(Ge_prime, Gestar_prime, Lg_prime, Lgstar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                  Ve_prime, Vestar_prime, Vgx1_prime, Vgx2_prime, Vgx_prime, Vgm_prime, Vg_prime,
                                  Vgstar_prime, chg_Qeworld, chg_consumption, chg_extraction, chg_production, delta_Le, 
                                  delta_Lestar, leakage1, leakage2, leakage3,
                                  pai_g, pe, subsidy_ratio, varphi, welfare, welfare_noexternality, jvals, consvals)

        diff, diff1, diff2 = comp_diff(consvals, jvals, Ge_prime, Gestar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                       Vgx2_prime, pe, tax_scenario, tb_mat, te, varphi, ParaList, df)

        obj_val = abs(diff1) + abs(diff) + abs(diff2)
        return obj_val

    ## retrive the pandas series object containing equilibrium values (prices, taxes, consumption etc)
    def retrive(self):
        ret = self.results
        ret['tb'] = self.tb
        ret['prop'] = self.prop
        ret['te'] = self.te
        ret['region_scenario'] = self.region
        ret['conv'] = self.conv
        
        return ret

In [6]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../../output/BaselineCarbon_2015.csv", index_col=['regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])

## choose which regional scenario to run (runs all if not executed)
#df=df.drop([2,3,4,5,6,7])  
df = df[df['region_scenario'] == 3]
df

,region_scenario,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
regionbase,,,,,,,,,,,,,,,,
OECD37 as Home,3,8.62549,23.6505,32.27599,11.29367,2.48754,0.91057,17.58421,13.78121,18.49478,12.20424,20.07175,32.27599,32.27599,0.049234,0.819498


In [7]:
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1]},index=[1,2, 3, 4, 5, 6, 7, 8])
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral'], 'Base':[0]},index=[1])
#df = pd.DataFrame({'Qe': 4.4800448, 'Qestar': 27.795946, 'Qeworld': 32.27599, 'CeHH': 4.598403, 'CeHF': 1.196111,'CeFH': 0.42160103,'CeFF': 26.059874,'Ce': 5.7945137,'Cestar': 26.481474,'Ge': 5.0200038,'Gestar': 27.255985,'Ceworld': 32.27599,'Geworld': 32.27599,'jxbar': 0.015920602214279298,'jmbar':  0.7935787194577492}, index = [1])

In [8]:
def temp(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    varphilist = np.arange(0,3,0.1)
    #varphilist = np.arange (0,10,0.1)
    output=[]
    prevtb = [1,1,1,1]
    for varphi in varphilist:    
        tax_df=df.apply(mintemp, axis=1, raw=False, args=(prevtb, varphi, tax_scenario, ParaList))
        prevtb = tax_df[['region_scenario','tb', 'pe', 'te', 'prop']]
        output.append(tax_df)
        print(varphi)
    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb
    print(tax_scenario['tax_sce'])
    return output

In [9]:
def mintemp(df, prevtb, varphi, tax_scenario, ParaList):
    pe = 1
    te = 0.5
    tb_mat = [0,0.5]
    #print(prevtb['region_scenario'])
    
    if (type(prevtb) != list):
        curr_region = prevtb[prevtb['region_scenario'] == df['region_scenario']]
        vals = curr_region.values
        #print(vals)
        tb_mat = [vals[0][1], vals[0][4]]
        pe = vals[0][2]
        te = vals[0][3]
    tax_temp = taxrow_eq(pe, te, tb_mat, df, tax_scenario, varphi, ParaList)
    tax_temp.opt_tax()
    ret = tax_temp.retrive()
    #print(ret)
    return ret

In [10]:
temp_df = tax_scenario.apply(temp, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))
#temp_df.head()

C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:448: RuntimeWarning: invalid value encountered in double_scalars
  leakage1 = -(Qestar_prime - df['Qestar']) / (Qeworld_prime - df['Qeworld'])
C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:449: RuntimeWarning: divide by zero encountered in double_scalars
  leakage2 = -(Gestar_prime - df['Gestar']) / (Qeworld_prime - df['Qeworld'])
C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:450: RuntimeWarning: invalid value encountered in double_scalars
  leakage3 = -(Cestar_prime - df['Cestar']) / (Qeworld_prime - df['Qeworld'])
C:\Users\Alex\Desktop\Tobin\sam\Optimal-Unilateral-Carbon-Policy\Data\codes\ces\ces_fun_util.py:450: RuntimeWarning: divide by zero encountered in double_scalars
  leakage3 = -(Cestar_prime - df['Cestar']) / (Qeworld_prime - df['Qeworld'])


0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
1.0
1.1
1.2000000000000002
1.3
1.4000000000000001
1.5
1.6
1.7000000000000002
1.8
1.9000000000000001
2.0
2.1
2.2
2.3000000000000003
2.4000000000000004
2.5
2.6
2.7
2.8000000000000003
2.9000000000000004
Unilateral


In [11]:
output_list=[]
for i in range(1,len(tax_scenario)+1):
    output_list.append(temp_df.loc[i])
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)
Outcomes

,tax_sce,regionbase,varphi,pe,tb,prop,te,jxbar_prime,jmbar_prime,j0_prime,...,chg_extraction,chg_production,chg_consumption,chg_Qeworld,pai_g,subsidy_ratio,welfare,welfare_noexternality,region_scenario,conv
0,Unilateral,OECD37 as Home,0.0,1.000000,0.000000,0.500000,0.0,0.049234,0.819498,0.049234,...,0.000000,-3.552714e-15,0.000000,0.000000,8.881784e-16,0.000000,1.082735e-13,1.082735e-13,3.0,1.0
1,Unilateral,OECD37 as Home,0.1,0.993692,0.039336,0.310340,0.1,0.049252,0.819498,0.048155,...,-0.074710,3.278288e-02,0.085064,-0.368548,-2.004957e-01,0.005903,9.414484e-02,5.403069e-02,3.0,1.0
2,Unilateral,OECD37 as Home,0.2,0.988417,0.078280,0.090405,0.2,0.049305,0.819498,0.047137,...,-0.137375,5.098336e-02,0.153864,-0.732830,-4.048822e-01,0.011740,2.263736e-01,6.684557e-02,3.0,1.0
3,Unilateral,OECD37 as Home,0.3,0.984112,0.116941,0.043632,0.3,0.049389,0.819498,0.046174,...,-0.188626,5.488365e-02,0.206761,-1.094164,-6.134443e-01,0.017512,3.964666e-01,3.918763e-02,3.0,1.0
4,Unilateral,OECD37 as Home,0.4,0.980732,0.155421,0.032856,0.4,0.049502,0.819498,0.045263,...,-0.228958,4.466847e-02,0.244024,-1.453833,-8.265121e-01,0.023219,6.043208e-01,-2.864246e-02,3.0,1.0
5,Unilateral,OECD37 as Home,0.5,0.978240,0.193817,0.026228,0.5,0.049642,0.819498,0.044398,...,-0.258732,2.041844e-02,0.265818,-1.813123,-1.044467e+00,0.028862,8.499654e-01,-1.367713e-01,3.0,1.0
6,Unilateral,OECD37 as Home,0.6,0.976615,0.232226,0.022121,0.6,0.049808,0.819498,0.043577,...,-0.278174,-1.790970e-02,0.272186,-2.173348,-1.267757e+00,0.034440,1.133548e+00,-2.857860e-01,3.0,1.0
7,Unilateral,OECD37 as Home,0.7,0.975846,0.270742,0.019159,0.7,0.049997,0.819498,0.042795,...,-0.287371,-7.049797e-02,0.263033,-2.535896,-1.496907e+00,0.039954,1.455356e+00,-4.767607e-01,3.0,1.0
8,Unilateral,OECD37 as Home,0.8,0.975939,0.309466,0.017367,0.8,0.050208,0.819498,0.042050,...,-0.286255,-1.377035e-01,0.238091,-2.902277,-1.732544e+00,0.045405,1.815822e+00,-7.113382e-01,3.0,1.0
9,Unilateral,OECD37 as Home,0.9,0.976915,0.348501,0.016109,0.9,0.050440,0.819498,0.041339,...,-0.274580,-2.200957e-01,0.196884,-3.274186,-1.975428e+00,0.050793,2.215534e+00,-9.918418e-01,3.0,1.0


In [17]:
rho = 0
alpha = 0.15
## define CES production function and its derivative
def g(p, rho = rho, alpha = alpha):
    if rho == 0:
        return alpha**(-alpha) * (1-alpha)**(-(1-alpha)) * p**alpha
    else:
        t1 = (1-alpha)**(1/(1-rho))
        t2 = alpha**(1/(1-rho)) * p**(-rho/(1-rho))
        return (t1 + t2)**(-(1-rho)/rho)
    
def gprime(p, rho = rho, alpha = alpha):
    if rho == 0:
        return (alpha/(1-alpha))**(1-alpha) * p**(-(1-alpha))
    else:
        t1 = (1-alpha)**(1/(1-rho))
        t2 = alpha**(1/(1-rho)) * p**(-rho/(1-rho))
        coef = alpha **(1/(1-rho)) * p**(-rho/(1-rho) - 1)
        return (t1 + t2)**(-(1-rho)/rho - 1) * coef
pe = 0.993692
tb = 0.039336
Ce_prime = 18.579844
g(pe + tb) / gprime(pe + tb) * Ce_prime

127.95666058421335

In [15]:
#outcome_t = Outcomes[Outcomes['regionbase'] == 'US as Home']
Outcomes[['pe','tb', 'Qe_prime', 'Qestar_prime', 'Ce_prime', 'Cestar_prime','Lg_prime','Lgstar_prime', 'Vg_prime', 'Vgstar_prime', 'delta_Le','delta_Lestar','welfare_noexternality']]

,pe,tb,Qe_prime,Qestar_prime,Ce_prime,Cestar_prime,Lg_prime,Lgstar_prime,Vg_prime,Vgstar_prime,delta_Le,delta_Lestar,welfare_noexternality
0,1.000000,0.000000,8.625490,23.650500,13.781210,18.494780,69.157360,113.739917,91.874733,123.298533,0.000000,0.000000,1.082735e-13
1,0.993692,0.039336,8.331652,23.575790,13.327598,18.579844,69.092172,113.743292,91.785215,123.326287,-0.283942,-0.074474,5.403069e-02
2,0.988417,0.078280,8.030035,23.513125,12.894515,18.648644,69.032095,113.740439,91.696943,123.350457,-0.555294,-0.136578,6.684557e-02
3,0.984112,0.116941,7.719953,23.461874,12.480285,18.701541,68.976722,113.731741,91.609779,123.371199,-0.813797,-0.187126,3.918763e-02
4,0.980732,0.155421,7.400615,23.421542,12.083352,18.738804,68.925681,113.717522,91.523577,123.388638,-1.059168,-0.226749,-2.864246e-02
5,0.978240,0.193817,7.071099,23.391768,11.702269,18.760598,68.878626,113.698047,91.438191,123.402863,-1.291102,-0.255912,-1.367713e-01
6,0.976615,0.232226,6.730315,23.372326,11.335676,18.766966,68.835233,113.673528,91.353465,123.413924,-1.509268,-0.274915,-2.857860e-01
7,0.975846,0.270742,6.376965,23.363129,10.982281,18.757813,68.795190,113.644121,91.269232,123.421836,-1.713305,-0.283893,-4.767607e-01
8,0.975939,0.309466,6.009469,23.364245,10.640843,18.732871,68.758196,113.609929,91.185312,123.426569,-1.902821,-0.282805,-7.113382e-01
9,0.976915,0.348501,5.625884,23.375920,10.310139,18.691664,68.723947,113.570996,91.101499,123.428039,-2.077383,-0.271405,-9.918418e-01


In [14]:
#Outcomes.to_csv('../../output_ces/global0.csv'.format(theta, case), header=True)